## Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

## Primera mirada

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.shape

In [ ]:
df.isna().sum()

Notar que en este caso:
- ```Image path``` : Contiene el path al archivo
- ```Insurance_company``` : Nombre de la compañía de seguros involucrada
- ```Cost_of_vehicle``` : Costo inicial del vehículo involucrado, en USD 
- ```Min_coverage``` : Monto de la cobertura mínima
- ```Max_coverage```: Monto de la cobertura máxima
- ```Expiry_date```: Fecha de expiración de la cobertura           
- ```Condition```: 0 si el auto está sin daños, 1 si está dañado 

Como criterio conservador, simplemente voy a omitir de la muestra los datos faltantes

In [ ]:
df.dropna(inplace =True)

In [ ]:
df[df.Amount!=0]['Amount'].hist(bins =20);

Lo primero que voy a hacer es expresar el daño en términos relativos al costo original del vehículo

In [ ]:
df['relative_amount'] = df['Amount']/df['Cost_of_vehicle']

Se nota asimetría de los montos relativos, hacia daños bajos

Por el momento, vamos a trabajar solo con los datos que tenemos OK

In [ ]:
df.relative_amount.hist(bins=30)

Voy a separar, momentáneamente, vehículos dañados de los que no lo están

In [ ]:
df_sin_danno = df[df['Amount']==0]
df_con_danno = df[df['Amount']!=0]

In [ ]:
df_sin_danno = df_sin_danno.assign(severidad='indemne')

In [ ]:
df_con_danno = df_con_danno.assign(severidad=pd.qcut(df.relative_amount,
                                                     q=[0,0.33,0.66,1],
                                                     labels=['leve','media','severa']))

In [ ]:
df_con_danno.severidad.value_counts()

In [ ]:
len(df[df.Amount==0])

In [ ]:
df = pd.concat([df_con_danno,df_sin_danno],axis=0) 

In [ ]:
df

A su vez, respecto de los montos de cobertura máxima y mínima, parece razonable expresar en términos relativos

In [ ]:
df['p_min_cov'] = df['Min_coverage']/df['Cost_of_vehicle']
df['p_max_cov'] = df['Max_coverage']/df['Cost_of_vehicle']

In [ ]:
df['Insurance_company'].value_counts().sort_index()

En una primera versión del modelo, codificaremos la compañía de seguros

In [ ]:
ins_cos=pd.get_dummies(df['Insurance_company'],prefix='Co')

In [ ]:
df = pd.concat([df,ins_cos],axis=1)

A su vez, vamos a encontrar los días restantes que el vehículo está bajo cobertura

In [ ]:
HOY = pd.to_datetime(datetime.date(2021,6,30))

In [ ]:
df['Expiry_date']=pd.to_datetime(df['Expiry_date'])

In [ ]:
df['dias_pendientes_cobertura']=(df['Expiry_date']-HOY).dt.days.astype(int)

Y vamos a escalar otras variables de interés

In [ ]:
mms = MinMaxScaler()

In [ ]:
df[['Cost_of_vehicle','dias_pendientes_cobertura']]=mms.fit_transform(df[['Cost_of_vehicle',
                                                                          'dias_pendientes_cobertura']])

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['Insurance_company','Min_coverage','Max_coverage','Expiry_date','Condition',
                   'Amount','relative_amount','p_min_cov'],inplace=True)

Por último, elimino las columnas que no resultan de interés (notar que la cobertura mínima es fija).

In [ ]:
df